In [2]:

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda


In [6]:
# 加载数据
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data,batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [7]:
# 创建模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        

In [8]:
model = NeuralNetwork()

### 超参数定义
- epoch: 迭代数据集的次数
- batch_size: 更新数据之前需要传入样本数量
- learning_rate： 每个批次/epoch 更新模型的参数

In [10]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

### 优化循环

- 训练循环：循环访问训练集并且尝试收敛到最佳的参数
- 验证/测试循环：循环访问测试数据集，用来检查模型的精度是否在提高

### 损失函数
损失函数用来表征模型训练结果与正确值之间的差距，所以模型训练的主要目的就是最小化损失函数的值。创建的损失函数：
- 回归任务： nn.MSELoss (均方差)
- 分类任务   nn.NLLLoss (负对数似然)
- 交叉熵： nn.CrossEntropyLoss 结合了nn.LogSoftmax 和 nn.NLLLoss



In [11]:
loss_fn = nn.CrossEntropyLoss()

### 优化器
优化器就是调整模型的参数使得减少每个训练步骤中的模型损失值的过程。优化器主要有：
- SGD
- ADAM
- RMSProp

训练循环中优化三个步骤：
- 调用重置模型参数的梯度值，每次迭代都显式将其归零：optimizer.zero_grad()
- 调用反向传播预测损失值：loss.backward()
- 通过梯度调整参数： optimizer.step()

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch,(X,y) in enumerate(dataloader):
        # 计算预测值与损失值
        pred = model(X)
        loss = loss_fn(pred, y)

        #反向传播三部曲
        # 将参数的梯度值重置为零，因为Pytorch 中的参数的梯度每次计算都会累加
        optimizer.zero_grad()
        # 反向传播：就是由损失函数反向传播计算出来的每个参数的梯度值，并且保存在相应的参数下面
        loss.backward()
        # 通过梯度值调整更新参数
        optimizer.step()

        # 打印相关的参数
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")




In [19]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in  dataloader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [20]:
# 进行训练

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epock {t+1}\n-----------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done")

Epock 1
-----------------------
loss: 2.169856  [    0/60000]
loss: 2.158634  [ 6400/60000]
loss: 2.097372  [12800/60000]
loss: 2.115595  [19200/60000]
loss: 2.066057  [25600/60000]
loss: 2.016772  [32000/60000]
loss: 2.043740  [38400/60000]
loss: 1.963690  [44800/60000]
loss: 1.988068  [51200/60000]
loss: 1.898363  [57600/60000]
Test Error: 
 Accuracy: 58.1%, Avg loss: 1.892623 

Epock 2
-----------------------
loss: 1.923162  [    0/60000]
loss: 1.893267  [ 6400/60000]
loss: 1.769507  [12800/60000]
loss: 1.813618  [19200/60000]
loss: 1.699503  [25600/60000]
loss: 1.662534  [32000/60000]
loss: 1.677342  [38400/60000]
loss: 1.578223  [44800/60000]
loss: 1.615656  [51200/60000]
loss: 1.495543  [57600/60000]
Test Error: 
 Accuracy: 62.1%, Avg loss: 1.511939 

Epock 3
-----------------------
loss: 1.570234  [    0/60000]
loss: 1.541874  [ 6400/60000]
loss: 1.385552  [12800/60000]
loss: 1.460296  [19200/60000]
loss: 1.336903  [25600/60000]
loss: 1.345981  [32000/60000]
loss: 1.348247  [384

In [22]:
# 保存模型参数
torch.save(model.state_dict(),'model/model_weight.pth')
# 保存模型的结构
torch.save(model, 'model/model.pth')

In [24]:
# 加载模型的参数
# 先要定义模型的结构
model_pre = NeuralNetwork()
model_pre.load_state_dict(torch.load('model/model_weight.pth'))
test_loop(test_dataloader, model_pre, loss_fn)

Test Error: 
 Accuracy: 72.3%, Avg loss: 0.752473 



In [25]:
# 直接加载整个模型包括模型的框架和参数
model_all = torch.load('model/model.pth')
test_loop(test_dataloader, model_all, loss_fn)

Test Error: 
 Accuracy: 72.3%, Avg loss: 0.752473 

